In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
from bayes_opt import BayesianOptimization
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

# Assume you have your data loaded as X and y


In [ ]:

# Bayesian Optimization
def xgb_bayesian_opt(max_depth, learning_rate, n_estimators, subsample, colsample_bytree):
    model = xgb.XGBRegressor(max_depth=int(max_depth),
                            learning_rate=learning_rate,
                            n_estimators=int(n_estimators),
                            subsample=subsample,
                            colsample_bytree=colsample_bytree,
                            random_state=42)
    model.fit(X_train, y_train)
    score = -mean_squared_error(y_test, model.predict(X_test))
    return score

xgb_bo = BayesianOptimization(xgb_bayesian_opt, {
    'max_depth': (4, 10),
    'learning_rate': (0.01, 0.3),
    'n_estimators': (100, 500),
    'subsample': (0.5, 1),
    'colsample_bytree': (0.5, 1)
})
xgb_bo.maximize(init_points=5, n_iter=25)
best_params_bo = xgb_bo.max['params']


In [ ]:

# Tree-structured Parzen Estimator (TPE)
def xgb_tpe(params):
    model = xgb.XGBRegressor(**params, random_state=42)
    model.fit(X_train, y_train)
    score = -mean_squared_error(y_test, model.predict(X_test))
    return {'loss': score, 'status': STATUS_OK}

space = {
    'max_depth': hp.quniform('max_depth', 4, 10, 1),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.3),
    'n_estimators': hp.quniform('n_estimators', 100, 500, 1),
    'subsample': hp.uniform('subsample', 0.5, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1)
}

trials = Trials()
best_params_tpe = fmin(xgb_tpe, space, algo=tpe.suggest, max_evals=25, trials=trials)
best_params_tpe = {
    'max_depth': int(best_params_tpe['max_depth']),
    'learning_rate': best_params_tpe['learning_rate'],
    'n_estimators': int(best_params_tpe['n_estimators']),
    'subsample': best_params_tpe['subsample'],
    'colsample_bytree': best_params_tpe['colsample_bytree']
}


In [ ]:

# Hyperband
from hyperopt import hp
from hyperopt.pyll.stochastic import sample
from sklearn.model_selection import cross_val_score

def xgb_hyperband(params):
    model = xgb.XGBRegressor(**params, random_state=42)
    scores = cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error')
    return {'loss': -np.mean(scores), 'status': STATUS_OK}


In [ ]:

space = {
    'max_depth': hp.quniform('max_depth', 4, 10, 1),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.3),
    'n_estimators': hp.quniform('n_estimators', 100, 500, 1),
    'subsample': hp.uniform('subsample', 0.5, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1)
}

best_params_hyperband = fmin(xgb_hyperband, space, algo=partial(tpe.suggest, n_startup_jobs=10), max_evals=25)
best_params_hyperband = {
    'max_depth': int(best_params_hyperband['max_depth']),
    'learning_rate': best_params_hyperband['learning_rate'],
    'n_estimators': int(best_params_hyperband['n_estimators']),
    'subsample': best_params_hyperband['subsample'],
    'colsample_bytree': best_params_hyperband['colsample_bytree']
}


In [ ]:

# Train final models with the best hyperparameters found
model_bo = xgb.XGBRegressor(**best_params_bo, random_state=42)
model_tpe = xgb.XGBRegressor(**best_params_tpe, random_state=42)
model_hyperband = xgb.XGBRegressor(**best_params_hyperband, random_state=42)

model_bo.fit(X_train, y_train)
model_tpe.fit(X_train, y_train)
model_hyperband.fit(X_train, y_train)